In [17]:
from typing import Dict, List
from datamodel import Order, OrderDepth, TradingState, Trade
import json
import math
import statistics

class Trader:
    def __init__(self):
        self.mid_prices = {
            "CROISSANTS": [],
            "JAMS": [],
            "DJEMBES": [],
            "PICNIC_BASKET1": []
        }

    POSITION_LIMITS = {
        "VOLCANIC_ROCK": 400,
        "VOLCANIC_ROCK_VOUCHER_9500": 200,
        "VOLCANIC_ROCK_VOUCHER_9750": 200,
        "VOLCANIC_ROCK_VOUCHER_10000": 200,
        "VOLCANIC_ROCK_VOUCHER_10250": 200,
        "VOLCANIC_ROCK_VOUCHER_10500": 200
    }
    
    # Voucher information: strike prices and a fixed time premium.
    VOUCHER_INFO = {
        "VOLCANIC_ROCK_VOUCHER_9500": {"strike": 9500, "time_premium": 200},
        "VOLCANIC_ROCK_VOUCHER_9750": {"strike": 9750, "time_premium": 200},
        "VOLCANIC_ROCK_VOUCHER_10000": {"strike": 10000, "time_premium": 200},
        "VOLCANIC_ROCK_VOUCHER_10250": {"strike": 10250, "time_premium": 200},
        "VOLCANIC_ROCK_VOUCHER_10500": {"strike": 10500, "time_premium": 200}
    }
    
    # Threshold for a voucher to be considered mispriced (in seashells).
    THRESHOLD = 50
    
    # Trade size per signal.
    TRADE_SIZE = 1

    def run(self, state: TradingState):
        orders = {}
        conversions = 0
        traderData = "simple_option_arbitrage_v1"
        
        # List of products required for fair value calculation.
        required = ["VOLCANIC_ROCK"] + list(self.VOUCHER_INFO.keys())
        for prod in required:
            if prod not in state.order_depths:
                print(f"{prod} order book missing. No trade.")
                return {}, conversions, traderData
        
        # 1. Compute the mid price for the underlying VOLCANIC_ROCK.
        vol_mid = self.get_mid_price(state.order_depths["VOLCANIC_ROCK"])
        if vol_mid is None:
            print("Unable to compute VOLCANIC_ROCK mid price. No trade.")
            return {}, conversions, traderData
        print(f"VOLCANIC_ROCK mid price = {vol_mid:.2f}")
        
        # 2. For each voucher, compute its market mid price and the fair value.
        for voucher, info in self.VOUCHER_INFO.items():
            strike = info["strike"]
            time_premium = info["time_premium"]
            fair_value = max(vol_mid - strike, 0) + time_premium
            voucher_mid = self.get_mid_price(state.order_depths[voucher])
            print(f"{voucher} mid price = {voucher_mid:.2f}, Fair Value = {fair_value:.2f}")
            
            # 3. Check mispricing.
            pos = state.position.get(voucher, 0)
            limit = self.POSITION_LIMITS[voucher]
            # If the voucher is undervalued, buy.
            if voucher_mid < fair_value - self.THRESHOLD:
                diff = fair_value - voucher_mid
                print(f"{voucher} is undervalued by {diff:.2f} seashells.")
                # Use the best ask price to buy.
                best_ask, ask_vol = self.get_best_ask(state.order_depths[voucher])
                if best_ask is not None and ask_vol is not None and ask_vol >= self.TRADE_SIZE:
                    if pos + self.TRADE_SIZE <= limit:
                        print(f"Placing BUY order for {voucher}: {self.TRADE_SIZE} unit at price {best_ask}")
                        orders.setdefault(voucher, []).append(Order(voucher, best_ask, self.TRADE_SIZE))
                    else:
                        print(f"Position limit reached for {voucher} (buy side).")
                else:
                    print(f"Insufficient ask volume for {voucher}.")
            
            # If the voucher is overvalued, sell.
            elif voucher_mid > fair_value + self.THRESHOLD:
                diff = voucher_mid - fair_value
                print(f"{voucher} is overvalued by {diff:.2f} seashells.")
                # Use the best bid price to sell.
                best_bid, bid_vol = self.get_best_bid(state.order_depths[voucher])
                if best_bid is not None and bid_vol is not None and bid_vol >= self.TRADE_SIZE:
                    if pos - self.TRADE_SIZE >= -limit:
                        print(f"Placing SELL order for {voucher}: {self.TRADE_SIZE} unit at price {best_bid}")
                        orders.setdefault(voucher, []).append(Order(voucher, best_bid, -self.TRADE_SIZE))
                    else:
                        print(f"Position limit reached for {voucher} (sell side).")
                else:
                    print(f"Insufficient bid volume for {voucher}.")
            else:
                print(f"{voucher}: No trading signal (mispricing within threshold).")
        
        return orders, conversions, traderData

    # Helper methods for computing mid price, best bid, best ask.

    def get_mid_price(self, od: OrderDepth):
        best_bid, _ = self.get_best_bid(od)
        best_ask, _ = self.get_best_ask(od)
        if best_bid is not None and best_ask is not None:
            return (best_bid + best_ask) / 2.0
        elif best_bid is not None:
            return best_bid
        elif best_ask is not None:
            return best_ask
        return None

    def get_best_bid(self, od: OrderDepth):
        if od.buy_orders:
            price = max(od.buy_orders.keys())
            return price, abs(od.buy_orders[price])
        return None, None

    def get_best_ask(self, od: OrderDepth):
        if od.sell_orders:
            price = min(od.sell_orders.keys())
            return price, abs(od.sell_orders[price])
        return None, None

ModuleNotFoundError: No module named 'datamodel'